In [ ]:
import numpy as np
import pandas as pd
import random
import time

class ProbabilisticSampling:
    def build_equiv_rel(self, datas, fd):
        epsilon = {}
        for attr in range(len(datas[0])):
            epsilon[attr] = {}

        #work on left fd
        for i in fds[:, 0]:
            dict = {}
            attData = datas[:,i]
            for j, d in enumerate(attData):
                if not dict:
                    dict[d] = {j}
                else:
                    if d not in dict:
                        dict[d] = {j}
                    else:
                        dict[d].add(j)
            epsilon[i] = dict

        #work on right fd
        fdRight = set()
        for (left, right) in fds:
            fdRight.add(right)
            dict = {}
            for val in epsilon[left]:
                if val and val != -1:
                    key = "v" + str(left) + str(val)
                    if len(epsilon[left][val]) > 0:
                        for idx in epsilon[left][val]:
                            if not dict:
                                dict[key] = {idx}
                            else:
                                if key not in dict:
                                    dict[key] = {idx}
                                else:
                                    dict[key].add(idx)
                    if dict:
                        if key not in epsilon[right]:
                            epsilon[right][key] = dict[key]

        tmplist = []
        right_dict = {}
        for r in fdRight:
            arr = epsilon[r].values()
            arr = list(arr)
            for i in range(len(arr)-1):
                for j in range(1, len(arr)):
                    if i == j:
                        continue
                    if arr[i] and any(val in arr[j] for val in arr[i] if arr[j]):
                        arr[j] |= set(arr[i])
                        arr[i] = None

            combined_eq_rel_indices = [a for a in arr if a is not None]

            sequential_no = 0
            for i, each_set in enumerate(combined_eq_rel_indices):
                realvalue = datas[random.sample(each_set, 1)[0]][r]
                use_real_value = True
                for row_index in each_set:
                    if realvalue != datas[row_index][r]:
                        use_real_value = False
                        break

                key = ""

                if use_real_value and realvalue != None:
                    key = realvalue
                else:
                    key = "v" + str(sequential_no)
                    sequential_no += 1

                right_dict[key] = each_set
            epsilon[r] = right_dict
            right_dict = {}
        return epsilon

    def is_clean(self, instance, epsilon):
        clean = True
        for attr in epsilon:
            for key in epsilon[attr].keys():
                if key and 'v' in key and len(epsilon[attr][key]) > 1:
                    return False
        return clean
    
    def gen_repair(self, instance, fds):
        cleancells = np.full([len(instance), len(instance[0])], None)

        no_of_val = 1

    #     Generate set indices
        index_set = set()
        for i in range(len(instance)):
            for j in range(1, len(instance[0])):
                index_set.add((i, j))
        prevepsilon = {}
        while None in cleancells and index_set:
            row, col = random.sample(index_set, 1)[0]
            index_set.remove((row, col))
            cleancells[row][col] = instance[row][col]
            if col not in fds:
                continue
            epsilon = self.build_equiv_rel(cleancells, fds)
            isclean = self.is_clean(cleancells, epsilon)
            
            if not isclean:
                belonged_key = None
                expected_values = set()
                for key, value in prevepsilon[col].items():
                    if row in value:
                        belonged_key = key
                        value.remove(row)
                        while None in value:
                            value.remove(None)
                        expected_values = value
                        break

                if belonged_key == None:
                    cleancells[row][col] = 'var' + str(no_of_val)
                    no_of_val += 1
                elif belonged_key == 'v0':
                    if len(expected_values) >= 1:
                        cleancells[row][col] = None
                        while not cleancells[row][col] and len(expected_values) > 0:
                            expected_row = expected_values.pop()
                            cleancells[row][col] = cleancells[expected_row][col]
                    else:
                        cleancells[row][col] = 'var' + str(no_of_val)
                        no_of_val += 1

            epsilon = self.build_equiv_rel(cleancells, fds)
            prevepsilon = epsilon
            
        cleancells_with_tuplieid = np.concatenate((instance[:,:1], cleancells[:,1:]), axis=1)
        return cleancells_with_tuplieid
    
    def get_multiple_repairs(self, datas, fds, no_of_instances):
        r_instances =  np.expand_dims(self.gen_repair(datas, fds), axis=0)
        for i in range(1, no_of_instances):
            repair = self.gen_repair(datas, fds)
            r_instances = np.concatenate((r_instances, [repair]), axis=0)
        return r_instances

    def sample_tuple_from_repair_instances(self, tupleid, no_of_samples, r_instances):
        sample_tuples = np.expand_dims(r_instances[np.random.randint(len(r_instances))][tupleid], axis=0)
        for i in range(1, no_of_samples):
            sample_tuple = r_instances[np.random.randint(len(r_instances))][tupleid]
            sample_tuples = np.concatenate((sample_tuples, [sample_tuple]), axis=0)
        return sample_tuples

    def calc_prob(self, samples):
        dicts = {}
        for x in samples:
            if x in samples:
                index = -1
                for i, val in enumerate(samples):
                    if np.all(samples[i]==x):
                        index = i
                        break

                if index not in dicts:
                    dicts[index] = 1
                else:
                    dicts[index] += 1

        output = []
        n = len(samples)
        for key in dicts:
            amount = dicts[key]
            new_tuple = np.concatenate((samples[key], [amount/n]), axis=0)
            if len(output) == 0:
                output = np.expand_dims(new_tuple, axis=0)
            else:
                output = np.concatenate((output, [new_tuple]), axis=0)
        return output
    
    def get_probabilistic_samples(self, raw_data, fds, no_of_instances, no_of_samples):
        repair_instances = self.get_multiple_repairs(raw_data, fds, no_of_instances)
        samples = []
        for tuple_id in np.squeeze(raw_data[:, :1], axis=1):
            sample_tups = self.sample_tuple_from_repair_instances(int(tuple_id)-1, no_of_samples, repair_instances)
            sample_tups_with_prob = self.calc_prob(sample_tups)
            if len(samples) == 0:
                samples = sample_tups_with_prob
            else:
                samples = np.concatenate((samples, sample_tups_with_prob), axis=0)
        return samples

data_from_file = np.loadtxt("income.train.txt.5k", delimiter=',', dtype='str', encoding='utf-8-sig')
ori_instance = data_from_file[:200]
ori_instance = np.concatenate((np.expand_dims(range(1, len(ori_instance)+1), axis=1), ori_instance), axis=1)

fds = np.array([[5,2]])
ps = ProbabilisticSampling()

no_of_instances = 10 #number of repair instances that we want to generate
no_of_samples = 20 #number of samples used to calculate the probability
start = time.time()
probabilistic_samples = ps.get_probabilistic_samples(ori_instance, fds, no_of_instances, no_of_samples)
print("running time:", time.time() - start)
np.savetxt("probabilistic_samples.csv", probabilistic_samples, delimiter=",", fmt='%s')